
On-Time

In [ ]:
# SQL:
# select
#   tasks.merchant_id,
#   tasks.env,
#   tasks.id as task_id,
#   wp.original_eta,
#   wp.checkin_time,
#   date_diff('minute',original_eta,checkin_time) as on_time
# from tasks
# join way_points as wp on tasks.id = wp.task_id and wp.date >= 20250101 and wp.checkin_time between date '2025-03-01' and date '2025-04-30' and wp.env = tasks.env
# join merchants on merchants.id = tasks.merchant_id and merchants.env = tasks.env and merchants.subscription_type = 17
# where tasks.date >= 20250101
#   and tasks.status = 4
#   and tasks.task_type_id <>3
#   and tasks.ended_time between date '2025-04-01' and date '2025-04-30'
#   and tasks.merchant_id <> 6
#   and tasks.is_billable = 1
#   and (
#       (tasks.task_type_id = 2 and wp.pickup_dropoff_option = 0) OR
#       (tasks.task_type_id in (4,5) and wp.pickup_dropoff_option in (1,2)) OR
#       (merchants.partner_id = 875 and wp.position = 1)
#       )
#   and original_eta is not null

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# |df1 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/cancelus2.csv')
# df2 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/cancelus3.csv')
# df3 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/canceleu2.csv')
# df4 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/canceleu3.csv')

In [ ]:
df1 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/ontimev1.csv',index_col= 'task_id').sample(frac=0.1)
df2 = pd.read_csv('/Users/along/Documents/Analysis/Benchmarks/ontimev2.csv',index_col= 'task_id').sample(frac=0.1)
accounts = pd.read_csv('/content/drive/MyDrive/Analysis/Data/Benchmark/accounts.csv')